In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_folder = "../input/g-research-crypto-forecasting/"
crypto_df = pd.read_csv(data_folder + 'train.csv')
asset_details_df = pd.read_csv(data_folder + 'asset_details.csv')
supp_train_df = pd.read_csv(data_folder + 'supplemental_train.csv')
expl_test = pd.read_csv(data_folder + 'example_test.csv')

In [ ]:
asset_details_df["Asset_Name"] = asset_details_df["Asset_Name"].str.replace(' ','_')
asset_details_df["Asset_Name"] = asset_details_df["Asset_Name"].str.replace('.','_')

In [ ]:
asset_details = asset_details_df.copy()
asset_details_df = asset_details_df.set_index("Asset_ID")

In [ ]:
#Create a dictionary of data frames assigned to each coin 
dataframes = {}
for asset_id, asset_name in zip(asset_details.Asset_ID, asset_details.Asset_Name):    
    keys = range(4)
    vars()[asset_name] = crypto_df[crypto_df["Asset_ID"]==asset_id].set_index("timestamp")
    dataframes[asset_id] = vars()[asset_name]

In [ ]:
cleaned_dataframes = {}
for i in list(dataframes):
    cleaned_dataframes[i] = dataframes[i].reindex(range(dataframes[i].index[0],dataframes[i].index[-1]+60,60),method='pad')

In [ ]:
#Reduce the datastes to ease the correlation calculation
from datetime import datetime
import time
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))
reduced_dataframes = {}
for i in list(cleaned_dataframes):
    #reduced_dataframes[i] = cleaned_dataframes[i].loc[totimestamp('01/01/2021'):totimestamp('01/05/2021')]
    reduced_dataframes[i] = cleaned_dataframes[i].iloc[-100:]

In [ ]:
def get_macd(df) :
    # Moving Average Convergence Divergence
    # The formula for MACD = 12-Period EMA − 26-Period EMA
    # Luckily, the Pandas DataFrame provides a function ewm(), which together with the mean-function can calculate the Exponential Moving Averages
    exp1 = df["Close"].ewm(span=12, adjust=False).mean()
    exp2 = df["Close"].ewm(span=26, adjust=False).mean()
    macd = exp1 - exp2
    ema_macd = macd.ewm(span=9, adjust=False).mean()
    df["MACD"] = macd
    df["MACD_Signal"] = ema_macd

In [ ]:
def get_rsi(df) :
    #Relative Strenght Index
    #measures the magnitude of recent price changes to evaluate overbought or oversold conditions in the price of a stock or other asset
    delta = df['Close'].diff()
    up = delta.clip(lower=0)
    down = -1*delta.clip(upper=0)
    ema_up = up.ewm(com=13, adjust=False).mean()
    ema_down = down.ewm(com=13, adjust=False).mean()
    rs = ema_up/ema_down
    df['RSI'] = 100 - (100/(1 + rs))

In [ ]:
def get_mfi(df) :
    #Money Flow Index
    #Calculated base on typical price : Typical price = (high + low + close) / 3
    Typical_price = (df['High']+df['Low']+df["Close"])/3
    period =  14 #The typical period used for MFI is 14 days
    money_flow = Typical_price * df['Volume']
    
    positive_flow = []
    negative_flow = []
    
    for j in range(1, len(Typical_price)):
        if Typical_price.iloc[j] > Typical_price.iloc[j-1]:
            positive_flow.append(money_flow.iloc[j-1])
            negative_flow.append(0)
        elif Typical_price.iloc[j] < Typical_price.iloc[j-1]:
            negative_flow.append(money_flow.iloc[j-1])
            positive_flow.append(0)
        
        else:
            positive_flow.append(0)
            negative_flow.append(0)
    
    positive_mf = []
    negative_mf = []

    for j in range(period-1, len(positive_flow)):
        positive_mf.append( sum(positive_flow[j + 1- period : j+1]))
    
    for j in range(period-1, len(negative_flow)):
        negative_mf.append( sum(negative_flow[j + 1- period : j+1]))
    MFI = 100 * (np.array(positive_mf) / (np.array(positive_mf) + np.array(negative_mf) ))
    df["MFI"] = 0 
    df["MFI"][14:] = MFI

In [ ]:
mfi_macd_rsi_dataframes = reduced_dataframes
for i in list(mfi_macd_rsi_dataframes) :
    get_macd(mfi_macd_rsi_dataframes[i])
    get_rsi(mfi_macd_rsi_dataframes[i])
    get_mfi(mfi_macd_rsi_dataframes[i])

In [ ]:
mfi_macd_rsi_dataframes[3]